## Importing packages

In [1]:
import pygame as pg #for visuals
import time
import os
import random #for randomly changing height of the hurdles
import neat
pg.font.init()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## Global variables

In [2]:
w_width = 600
w_height = 750
gen=0

#### Importing all the required images from the imgs folder to make a game frame.

In [3]:
planeimgs = [pg.image.load(os.path.join("imgs","planeYellow1.png")),
             pg.image.load(os.path.join("imgs","planeYellow2.png")),
             pg.image.load(os.path.join("imgs","planeYellow3.png"))] 
            #scale2x() makes the images twice the original size
            #load() loads the images

In [4]:
hurdleimg = pg.transform.scale2x(pg.image.load(os.path.join("imgs","rockGrass.png")))

In [5]:
baseimg = pg.transform.scale2x(pg.image.load(os.path.join("imgs","groundGrass.png")))

In [6]:
bgimg = pg.transform.scale2x(pg.image.load(os.path.join("imgs","background.png")))

stat_font=pg.font.SysFont("comicsans",50)

## Plane class

In [7]:
class Plane:
    maxrot = 25  #maximum rotation - how much the plane may tilt
    rotvel = 20  #how much to rotate each frame when the plane is rotated
    anitime = 5  #how long to show each plane animation
    imgg = planeimgs
    
    def __init__(self, x, y):
         
            #starting positions of the plane
        self.x = x
        self.y = y
        
        self.tilt = 0  #how much the plane is actually titled - 0 initially because we want it to start at a horizontal position
        self.tick_count = 0 #get to know how high or low the plane, when we last jumped
        self.vel = 0
        self.height = self.y
        self.img_count = 0 #so as to know which image is currently being displayed
        self.imgs = planeimgs[0] #basically first plane image
        
    def jump(self):
        self.vel = -10.5 #negative because in CG negative represents upward while positive represents downwards
        self.tick_count = 0 #set to 0 because we need to know when we are changing directions/velocities
        self.height = self.y #to keep track where the plane last jumped from
    
    def move(self): 
        self.tick_count += 1 #how many times we moved since the last jump
                #by moved, I mean the frame changed
            
        d = self.vel*(self.tick_count) + 1.5*(self.tick_count**2)  # calculate displacement
        
        #terminal velocity
        if d>=16:     #to not go less than 16px downwards
            d = (d/abs(d)) * 16
        
        if d<0 : 
            d -= 2  #to not jump abruptly
            
        self.y = self.y + d
        
        if d<0 or self.y<self.height+50 :  #if we are moving upwards 
            if self.tilt < self.maxrot: #so that we don't tilt the plane to some unexpected direction
                self.tilt = self.maxrot
                
        else:
            if self.tilt > -90:
                self.tilt -= self.rotvel  #rotating the plane completely downwards(90) 
    
    def draw(self, win):
        
        self.img_count += 1
        
        #deciding which plane image to show based on the time elapsed
        if self.img_count < self.anitime:
            self.imgs = planeimgs[0]
        elif self.img_count < self.anitime*2:
            self.imgs = planeimgs[1]
        elif self.img_count < self.anitime*3:
            self.imgs = planeimgs[2]
        elif self.img_count < self.anitime*4:
            self.imgs = planeimgs[1]
        elif self.img_count == self.anitime*4 + 1:
            self.imgs = planeimgs[0]
            self.img_count = 0
            
        # so when bird is nose diving it isn't flapping
        if self.tilt <= -80:
            self.imgs = planeimgs[1]
            self.img_count = self.anitime*2
            
        #rotating the plane
        rimg = pg.transform.rotate(self.imgs,self.tilt)
        n_rect = rimg.get_rect(center = self.imgs.get_rect(topleft = (self.x,self.y)).center)
        win.blit(rimg,n_rect.topleft)
        
    def get_mask(self):
        return pg.mask.from_surface(self.imgs)
    
    def xy(self):
        return type(self.imgs)

## Rock Class

In [8]:
class Rock:
    gap=-190 #space between top and bottom rock
    vel=5 #speed with which the rock must move toward the plane
    
    def __init__(self,x):
        self.x=x
        self.height=0 #height of the rock
        self.top=0 #length of the top rock
        self.bottom=0 #length of the bottom rock
        self.rocktop=pg.transform.flip(hurdleimg,False,True) #flips for the upside dowm image
        self.rockbottom=hurdleimg
        self.passed= False #if the bird has already passed by the obstacle/collision purposes
        self.set_height()
    
    def set_height(self):  #getting the position where our rocks should appear
        self.height=random.randrange(200,400)
        self.top=self.height-self.rocktop.get_height()
        self.bottom=self.height-self.gap
    
    def move(self): #moving the rocks toward the left
        self.x -= self.vel
    
    def draw(self,win):
        win.blit(self.rocktop,(self.x,self.top))
        win.blit(self.rockbottom,(self.x,self.bottom))
    
    def collide(self,plane): #checking if the top and bottom rocks are colliding or the rocks and plane are colliding
        
        #getting the masks of each of the elements under comparison
        plane_mask=plane.get_mask()
        top_mask=pg.mask.from_surface(self.rocktop)
        bottom_mask=pg.mask.from_surface(self.rockbottom)
        
        #offsets represent how far these masks are from each other
        top_offset=(self.x-plane.x,self.top-round(plane.y)) #offset of the plane from the top mask
        bottom_offset=(self.x-plane.x,self.bottom-round(plane.y))
        
        #to find if the masks are colliding
        b_point=plane_mask.overlap(bottom_mask,bottom_offset)
        t_point=plane_mask.overlap(top_mask,top_offset)
        
        if t_point or b_point:
            return True #collision is occuring
        return False
    

## Ground Class

In [9]:
class Ground:
    vel=5
    width=baseimg.get_width()
    img=baseimg

    def __init__(self,y):
        self.y=y
        self.x1=0
        self.x2=self.width
        
    def move(self):
        self.x1 -= self.vel
        self.x2 -= self.vel
        
        #for a continuously moving ground effect, we use two ground images that are displayed one after the other in a loop
        #if an image is off the screen, we cycle it back behind the next image and vice versa
        if self.x1+self.width<0 :
            self.x1= self.x2+self.width
        
        if self.x2+self.width<0 :
            self.x2=self.x1+self.width
            
    def draw(self,win):
        win.blit(self.img,(self.x1,self.y))
        win.blit(self.img,(self.x2,self.y))
        

## Main functions for displaying the work above

In [10]:
def draw_window(win,planes,rocks,base,score,gen):
    win.blit(bgimg, (0,0)) #blit just draws
    
    for rock in rocks:
        rock.draw(win)
    
    text= stat_font.render("Score: " + str(score),1,(255,255,255))
    win.blit(text, (w_width - 10 - text.get_width(),10))
    
    text= stat_font.render("Generation: " + str(gen),1,(255,255,255))
    win.blit(text, (10,10))
    
    base.draw(win)
    
    for plane in planes:
        plane.draw(win)
    pg.display.update()

In [11]:
def main(genomes, config):
    global gen
    gen+=1
    nets=[]
    ge=[]
    
    planes = []
    
    for _,g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        planes.append(Plane(230,350))
        g.fitness = 0
        ge.append(g)
    
    base = Ground(650)
    rocks=[Rock(730)]
    win = pg.display.set_mode((w_width,w_height))
    
    clock = pg.time.Clock()

    score=0
    
    run = True
    
    while run:
        
        clock.tick(25) #the more the value here, the slower frames will change per second
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
                pg.quit()
                quit() 
                
        rock_ind = 0
        if len(planes)>0:
            if len(rocks)>1 and planes[0].x > rocks[0].x + rocks[0].rocktop.get_width():
                rock_ind = 1
        else:
            run=False
            break
                
        for x,plane in enumerate(planes):
            plane.move()
            ge[x].fitness += 0.1
            
            output = nets[planes.index(plane)].activate((plane.y, abs(plane.y - rocks[rock_ind].height), abs(plane.y - rocks[rock_ind].bottom)))
                    
            if output[0]>0.5:
                plane.jump()
                
        #plane.move() 
        add_rock=False
        rem=[]
        
        for rock in rocks:
            
            for x,plane in enumerate(planes):
                
                if rock.collide(plane):
                    ge[x].fitness -= 1 #everytime a bird hits a pipe, 1 is decreased from its fitness score
                    planes.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                    
                if not rock.passed and rock.x < plane.x:
                    rock.passed=True
                    add_rock=True 
            
            if rock.x+rock.rocktop.get_width()<0: #if the rock is completeley off the screen
                    rem.append(rock)
            
            rock.move()
            
        if add_rock:
            score+=1
            for g in ge:
                g.fitness += 5 #giving the planes that have reached the next level an increment of 5 in their fitness score
            rocks.append(Rock(730))
    
        for r in rem:
            rocks.remove(r)
            
        for x,plane in enumerate(planes):
            if plane.y+plane.imgs.get_height() >= 730 or plane.y<0: #if the plane hits the ground or jumps higher than the limit
                planes.pop(x)
                nets.pop(x)
                ge.pop(x)
                
        if score>50:
            break
        
        
        base.move()
        draw_window(win,planes,rocks,base,score,gen)  

## NEAT Algorithm implementation

**Input** - position of the plane (just the y coordinate), coordinate of the top pipe and the bottom pipe

**Output** - jump or don't jump

**Activation function** - tan h (hyperbolic tangent) ranges from (-1,1)

**Population size** (how many planes flying in each generation) - 150 planes

**Fitness function** - the farther the plane is, the more it scored

**Max number of generation** - 30

In [12]:
def run(config_path):
    #giving it the properties to set
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                                neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path) 
    
    p = neat.Population(config)
    
    #to create visuals
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    winner = p.run(main,50)

In [13]:
if __name__ == '__main__':
#     local_dir = os.path.dirname(__file__) #gives us the path of the directory we currently are in
    config_path = os.path.join("config_feedforward.txt") 
    run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 6.12133 stdev: 8.26955
Best fitness: 76.20000 - size: (1, 3) - species 1 - id 30
Average adjusted fitness: 0.054
Mean genetic distance 1.078, standard deviation 0.427
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150     76.2    0.054     0
Total extinctions: 0
Generation time: 23.440 sec

 ****** Running generation 1 ****** 

Population's average fitness: 14.22667 stdev: 63.34127
Best fitness: 775.20000 - size: (2, 4) - species 1 - id 267

Best individual in generation 1 meets fitness threshold - complexity: (2, 4)
